#Voice Chat Bot

##Installing modules

In [1]:
!apt install python3-pyaudio
!pip install SpeechRecognition
!pip install gTTS
!pip install wikipedia
#!pip install webbrowser
!pip -q install pydub

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libportaudio2
Suggested packages:
  python-pyaudio-doc
The following NEW packages will be installed:
  libportaudio2 python3-pyaudio
0 upgraded, 2 newly installed, 0 to remove and 19 not upgraded.
Need to get 89.8 kB of archives.
After this operation, 336 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libportaudio2 amd64 19.6.0-1build1 [65.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 python3-pyaudio amd64 0.2.11-1.1build1 [24.4 kB]
Fetched 89.8 kB in 0s (278 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 128208 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1build1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1build1) ...
Selecting previously unselected package python3-pyau

##Importing Modules

In [2]:
import speech_recognition as sr
import pyaudio

from gtts import gTTS
from IPython.display import Audio
from IPython.core.display import display
from pydub import AudioSegment

import wikipedia
import webbrowser

import time

##Voice recognizer(Voice to text)

In [3]:
r = sr.Recognizer()
r.energy_threshold = 300

In [4]:
len(sr.Microphone.list_microphone_names())

0

###Getting Access to microphone (javascript)

In [5]:
#Only for Colab(web browser)
# all imports
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
from io import BytesIO

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=5):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  audio = AudioSegment.from_file(BytesIO(b))
  return audio

###Voice to text (Microphone)

In [6]:
#voice recognition from mic
def listen():
  with sr.Microphone() as source:
      speak("Please say anything")
      print("Listening....")
      audio = r.listen(source)
      try:
          text = r.recognize_google(audio)
          return("You said : {}".format(text))
      except:
          return("Sorry could not recognize what you said")

##Voice to text (Using Audio file)

In [7]:
#Creating Demo voice Files
def createAudioFile(text):
  tts = gTTS(text) #Provide the string to convert to speech
  tts.save('1.wav') #save the string converted to speech as a .wav file(Saves to MPEG format)
  #sound_file = '1.wav'
  #Audio(sound_file, autoplay=True)
  sound = AudioSegment.from_mp3("1.wav")
  sound.export("s1.wav", format="wav")
  return ("s1.wav")

In [8]:
def voiceFileToText(path):
  audioFile = sr.AudioFile(path)
  with audioFile as source:
    audio = r.record(source)
    txt  = r.recognize_google(audio)
  return txt

## Text To Voice


In [9]:
def audioDuration(fileName):
  #loading audio file form our system
  sound = AudioSegment.from_file(fileName)
  #duration calculation function
  if(sound.duration_seconds == (len(sound) / 1000.0)):
    #seconds to minutes conversion
    minutes_duartion = int(sound.duration_seconds // 60)
    seconds_duration = round((sound.duration_seconds % 60),3)
    #print(int(sound.duration_seconds ))
    #print(minutes_duartion,':',seconds_duration)
  return sound.duration_seconds
  #return minutes_duartion*60+seconds_duration

In [10]:
#
def speak(text):
  tts = gTTS(text, lang = language) #Provide the string to convert to speech
  tts.save('2.wav') #save the string converted to speech as a .wav file(Saves to MPEG format)
  sound_file = '2.wav'
  display(Audio(sound_file, autoplay=True))
  print(text)
  time.sleep(audioDuration(sound_file) +0.3)


##Voice Bot 


In [25]:
def voiceBot(language):

  botName = "Vinu"
  bossName = ""
  wikipedia.set_lang(language)

  if language =='en':
    speak("My name is "+ botName)
    speak("I am here to help you")

  if language =='hi':
    speak("मेरा नाम है "+ botName)
    speak("मैं आपकी कैसे मदद कर सकता हूँ")  
  while(True):
    #speech to text
    if (len(sr.Microphone.list_microphone_names())!=0):
      statement = listen() #if mic Available
    else :
      if language =='en':
        speak("Please type your query")
      if language =='hi':
        speak("प्लीज अपना सवाल लिखिए")
      textInput = input()
      speechFilePath = createAudioFile(textInput)
      statement = voiceFileToText(speechFilePath)
    
    #Processing speechText
    if 'hello' in statement:
      speak("hi")

    elif 'what is your name' in statement:
      speak("My name is "+ botName)

    elif 'My name is' in statement:
      statement = statement.replace("My name is", "")
      bossName = statement
      speak("Ok your name is "+ bossName) 

    elif 'what is' in statement:
      statement = statement.replace("what is", "")
      results = wikipedia.summary(statement, sentences=2)
      #speak("According to Wikipedia")
      print(results)
      speak(results)
    
    elif 'who is' in statement:
      statement = statement.replace("who is", "")
      results = wikipedia.summary(statement, sentences=2)
      #speak("According to Wikipedia")
      speak(results)

    elif 'where is' in statement:
      #statement = statement.replace("where is", "")
      results = wikipedia.summary(statement, sentences=1)
      #speak("According to Wikipedia")
      speak(results)

    elif "good bye" in statement or "okay bye" in statement or "stop" in statement:
      if language=='en':
        speak('your personal assistant '+botName+' is shutting down,Good bye')
      if language=='hi':
        speak('आपका पर्सनल असिस्टेंट '+botName+' बंद हो रहा है, धन्यवाद्')
      break 
    else : 
      if language=='en':
        speak("sorry, i did not found any result")
      if language=='hi':
        speak("क्षमा करें, मुझे कोई परिणाम नहीं मिला")
    # elif 'open youtube' in statement:
    #   webbrowser.open_new_tab("https://www.youtube.com")
    #   speak("youtube is open now")
    #   time.sleep(5)



In [27]:
language = 'hi'
voiceBot(language)

मेरा नाम है Vinu


मैं आपकी कैसे मदद कर सकता हूँ


प्लीज अपना सवाल लिखिए
What is Mathmatics
result2:
{   'alternative': [   {   'confidence': 0.97219545,
                           'transcript': 'what is mathematics'}],
    'final': True}
गणित ऐसी विद्याओं का समूह है जो संख्याओं, मात्राओं, परिमाणों, रूपों और उनके आपसी रिश्तों, गुण, स्वभाव इत्यादि का अध्ययन करती हैं। गणित एक अमूर्त या निराकार (abstract) और निगमनात्मक प्रणाली है। गणित की कई शाखाएँ हैं : अंकगणित, रेखागणित, त्रिकोणमिति, सांख्यिकी, बीजगणित, कलन, इत्यादि। गणित में अभ्यस्त व्यक्ति या खोज करने वाले वैज्ञानिक को गणितज्ञ कहते हैं।
बीसवीं शताब्दी के प्रख्यात ब्रिटिश गणितज्ञ और दार्शनिक बर्टेंड रसेल के अनुसार ‘‘गणित को एक ऐसे विषय के रूप में परिभाषित किया जा सकता है जिसमें हम जानते ही नहीं कि हम क्या कह रहे हैं, न ही हमें यह पता होता है कि जो हम कह रहे हैं वह सत्य भी है या नहीं।’’ 
गणित कुछ अमूर्त धारणाओं एवं नियमों का संकलन मात्र ही नहीं है, बल्कि दैनंदिन जीवन का मूलाधार है।


== गणित का महत्व ==

पुरातन काल से ही सभी प्रकार के ज्ञान-विज्ञान में गणित का स्थान सर्वोपरि रहा है-

यथ

KeyboardInterrupt: ignored